In [1]:
import requests
import time 
import numpy as np
import pandas as pd
import copy

In [ ]:
def collect_data(video_id):
    video_dictionary = {}
    json_data = {
        'context': {
            'client': {
                'clientName': 'WEB',
                'clientVersion': '2.20231208.01.00',            
                }
        },
        'videoId': video_id,
    } # collect for post-identification of the latter (not the former!)
    first_response = requests.post('https://www.youtube.com/youtubei/v1/next', json=json_data)
    second_response = requests.post('https://www.youtube.com/youtubei/v1/player', json=json_data)
    video_response = first_response.json()
    meta_response = second_response.json()
    individual_video_data = video_response["contents"]["twoColumnWatchNextResults"]["results"]["results"]["contents"]
    video_dictionary["videoId"] = video_id
    video_dictionary["exactViewNumber"] = individual_video_data[0]["videoPrimaryInfoRenderer"]["viewCount"]["videoViewCountRenderer"]["viewCount"]["simpleText"]
    video_dictionary["numberLikes"] = individual_video_data[0]["videoPrimaryInfoRenderer"]["videoActions"]["menuRenderer"]["topLevelButtons"][0]["segmentedLikeDislikeButtonViewModel"]["likeButtonViewModel"]["likeButtonViewModel"]["toggleButtonViewModel"]["toggleButtonViewModel"]["defaultButtonViewModel"]["buttonViewModel"]["title"]
    video_dictionary["creatorSubscriberNumber"] = individual_video_data[1]["videoSecondaryInfoRenderer"]["owner"]["videoOwnerRenderer"]["subscriberCountText"]["simpleText"]
    try:    
        video_dictionary["numberOfComments"] = individual_video_data[2]["itemSectionRenderer"]["contents"][0]["commentsEntryPointHeaderRenderer"]["commentCount"]["simpleText"]
    except KeyError: # correct for additional "merchandise" section
        try:
            video_dictionary["numberOfComments"] = individual_video_data[3]["itemSectionRenderer"]["contents"][0]["commentsEntryPointHeaderRenderer"]["commentCount"]["simpleText"]
        except IndexError: # comments are turned off
            video_dictionary["numberOfComments"] = np.nan
    try:
        video_dictionary["isCreatorVerified"] = (individual_video_data[1]["videoSecondaryInfoRenderer"]["owner"]["videoOwnerRenderer"]["badges"][0]["metadataBadgeRenderer"]["tooltip"] == "Verified")
    except KeyError:
        video_dictionary["isCreatorVerified"] = False
    try:
        video_dictionary["videoKeywords"] = meta_response["videoDetails"]["keywords"]
    except KeyError: # no keywords
        video_dictionary["videoKeywords"] = np.nan
    video_dictionary["videoLengthSeconds"] = meta_response["videoDetails"]["lengthSeconds"]
    video_dictionary["videoCategory"] = meta_response["microformat"]["playerMicroformatRenderer"]["category"]
    video_dictionary["videoExactPublishDate"] = meta_response["microformat"]["playerMicroformatRenderer"]["uploadDate"]
    return None
